# Spark - run SQL queries

* ***원래 소스 가지고 안돌아가네요. 저자가 testing 하고 Community 에 올렸을때와 달리 개인 환경에 변화가 있었을 거라고 생각됩니다.***
* ***pandas로 box 에 있는 화일을 읽어서, Spark SQL로  query를 돌려봅니다.***



## Table of contents
This notebook contains these main sections:

1. [Load a DataFrame](#Load_a_DataFrame)
2. [Create an SQLContext](#Create_an_SQLContext)
3. [Create a Spark DataFrame](#Create_a_Spark_DataFrame)
4. [Aggregate data after grouping by columns](#Aggregate_data_after_grouping_by_columns)
5. [Operate on columns](#Operate_on_columns)
6. [Run SQL queries from the Spark DataFrame](#Run_SQL_queries_from_the_Spark_DataFrame)

In [5]:
!pip install pyspark

    100% |████████████████████████████████| 211.9MB 4.2kB/s eta 0:00:01 2% |█                               | 6.3MB 46.4MB/s eta 0:00:05    56% |██████████████████              | 118.7MB 50.6MB/s eta 0:00:0201�██████████▊      | 170.6MB 47.2MB/s eta 0:00:01��████████▏     | 173.0MB 48.4MB/s eta 0:00:01████████▋     | 176.1MB 47.3MB/s eta 0:00:01��██████     | 178.3MB 49.7MB/s eta 0:00:01    | 183.8MB 48.0MB/s eta 0:00:010:00:01████████████████████████▍   | 188.0MB 47.1MB/s eta 0:00:01�██▊   | 190.0MB 46.9MB/s eta 0:00:01�████████████████████████████   | 191.9MB 48.6MB/s eta 0:00:01B/s eta 0:00:01.0MB 49.9MB/s eta 0:00:01████████████████  | 198.0MB 47.5MB/s eta 0:00:01.2MB/s eta 0:00:01 95% |██████████████████████████████▌ | 201.7MB 50.6MB/s eta 0:00:011�███████████████████████████ | 205.8MB 49.7MB/s eta 0:00:01�| 208.2MB 53.2MB/s eta 0:00:011
    100% |████████████████████████████████| 194kB 4.9MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyspark ... done
  Stored in directory: 

<a id='Load_a_DataFrame'></a>
## 1. Load a DataFrame
A DataFrame is a distributed collection of data that is organized into named columns. The Python pandas DataFrame that you will create will load an external DataFrame called **mtcars**, which includes observations on the following 11 variables:

`[, 1]	mpg     Miles / (US) gallon`  
`[, 2]	cyl     Number of cylinders`  
`[, 3]	disp	Displacement (cu. in.)`  
`[, 4]	hp      Gross horsepower`  
`[, 5]	drat    Rear axle ratio`  
`[, 6]	wt      Weight (1000 lbs)`  
`[, 7]	qsec    1/4 mile time (seconds)`  
`[, 8]	vs      0 = V-engine, 1 = straight engine`  
`[, 9]	am      Transmission (0 = automatic, 1 = manual)`  
`[,10]	gear    Number of forward gears`  
`[,11]	carb    Number of carburetors`


* ***library 추가 했습니다.***

In [18]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import pyspark.sql 

In [7]:
import pandas as pd
mtcars = pd.read_csv('https://ibm.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv')

Preview the first 3 rows of the DataFrame by using the `head()` method:

In [15]:
mtcars.head(3)

,car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1


<a id='Create_an_SQLContext'></a>
## 2. Create an SQLContext
To work with a DataFrame, you need an SQLContext class object, and to create a basic one, all you need is a SparkContext. A SparkContext represents the connection to a Spark cluster, and a SparkContext class object named sc, which has been created for you, is used to initialize the SQLContext:

* ***library import 하고 나서도 sc 때문에 에러나서 코드 2줄 추가했습니다.***

In [19]:
conf = SparkConf().setAppName("Spark SQL")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

<a id='Create_a_Spark_DataFrame'></a>
## 3. Create a Spark DataFrame
Using the SQLContext class object and the loaded local DataFrame, create a Spark DataFrame and print the schema, or structure, of the DataFrame:

In [20]:
sdf = sqlContext.createDataFrame(mtcars) 
sdf.printSchema()

root
 |-- car: string (nullable = true)
 |-- mpg: double (nullable = true)
 |-- cyl: long (nullable = true)
 |-- disp: double (nullable = true)
 |-- hp: long (nullable = true)
 |-- drat: double (nullable = true)
 |-- wt: double (nullable = true)
 |-- qsec: double (nullable = true)
 |-- vs: long (nullable = true)
 |-- am: long (nullable = true)
 |-- gear: long (nullable = true)
 |-- carb: long (nullable = true)



* ***Spark DataFrame 에 넣오 놓고 10 row 만 출력해봅니다.***

In [28]:
sdf.show(10)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              car| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|        Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|    Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|
|          Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|
|       Duster 360|14.3|  8|360.0|245|3.21| 3.57|15.84|  0|  0|   3|   4|
|        Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2|
|         Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|
|         Merc 280|19.2|  6|167.6|123|3.92| 3.44| 18.3|  1|  0|   4|   4|
+-----------------+----+---+-----+---+

* ***mpg 칼럼만 query 해서 보여줍니다.***

In [29]:
sdf.select('mpg').show(5)

+----+
| mpg|
+----+
|21.0|
|21.0|
|22.8|
|21.4|
|18.7|
+----+
only showing top 5 rows



* ***where 조건도 한번 줘 보고요***

In [23]:
sdf.filter(sdf['mpg'] < 18).show()

+-------------------+----+---+-----+---+----+------------------+-----+---+---+----+----+
|                car| mpg|cyl| disp| hp|drat|                wt| qsec| vs| am|gear|carb|
+-------------------+----+---+-----+---+----+------------------+-----+---+---+----+----+
|         Duster 360|14.3|  8|360.0|245|3.21|              3.57|15.84|  0|  0|   3|   4|
|          Merc 280C|17.8|  6|167.6|123|3.92|              3.44| 18.9|  1|  0|   4|   4|
|         Merc 450SE|16.4|  8|275.8|180|3.07|              4.07| 17.4|  0|  0|   3|   3|
|         Merc 450SL|17.3|  8|275.8|180|3.07|              3.73| 17.6|  0|  0|   3|   3|
|        Merc 450SLC|15.2|  8|275.8|180|3.07|              3.78| 18.0|  0|  0|   3|   3|
| Cadillac Fleetwood|10.4|  8|472.0|205|2.93|              5.25|17.98|  0|  0|   3|   4|
|Lincoln Continental|10.4|  8|460.0|215| 3.0|5.4239999999999995|17.82|  0|  0|   3|   4|
|  Chrysler Imperial|14.7|  8|440.0|230|3.23|             5.345|17.42|  0|  0|   3|   4|
|   Dodge Challenger|

<a id='Aggregate_data_after_grouping_by_columns'></a>
## 4. Aggregate data after grouping by columns
* ***cyl 칼럼으로 gropu by 해서  wt 값을 평균냅니다.***

In [31]:
sdf.groupby(['cyl']).agg({"wt": "AVG"}).show()

+---+------------------+
|cyl|           avg(wt)|
+---+------------------+
|  6| 3.117142857142857|
|  8|3.9992142857142867|
|  4| 2.285727272727273|
+---+------------------+



In [32]:
sdf.groupby(['cyl']).agg({"wt": "SUM"}).show()

+---+-----------------+
|cyl|          sum(wt)|
+---+-----------------+
|  6|            21.82|
|  8|55.98900000000001|
|  4|           25.143|
+---+-----------------+



* ***Count 내서 descending sort 합니다.***

In [33]:
car_counts = sdf.groupby(['cyl']).agg({"wt": "count"}).sort("count(wt)", ascending=False).show()

+---+---------+
|cyl|count(wt)|
+---+---------+
|  8|       14|
|  4|       11|
|  6|        7|
+---+---------+



<a id='Operate_on_columns'></a>
## 5. Operate on columns
Python provides a number of functions that you can apply directly to columns for data processing. In the following example, a basic arithmetic function converts lbs to metric tons:

In [26]:
sdf.withColumn('wtTon', sdf['wt'] * 0.45).select('car','wt','wtTon').show(6)

+-----------------+-----+-------+
|              car|   wt|  wtTon|
+-----------------+-----+-------+
|        Mazda RX4| 2.62|  1.179|
|    Mazda RX4 Wag|2.875|1.29375|
|       Datsun 710| 2.32|  1.044|
|   Hornet 4 Drive|3.215|1.44675|
|Hornet Sportabout| 3.44|  1.548|
|          Valiant| 3.46|  1.557|
+-----------------+-----+-------+
only showing top 6 rows



<a id='Run_SQL_queries_from_the_Spark_DataFrame'></a>
## 6. Run SQL queries from the Spark DataFrame
You can register a Spark DataFrame as a temporary table and then run SQL queries over the data. The `sql` function enables an application to run SQL queries programmatically and returns the result as a DataFrame:

In [27]:
sdf.registerTempTable("cars")

highgearcars = sqlContext.sql("SELECT car, gear FROM cars WHERE gear >= 5")
highgearcars.show()    

+--------------+----+
|           car|gear|
+--------------+----+
| Porsche 914-2|   5|
|  Lotus Europa|   5|
|Ford Pantera L|   5|
|  Ferrari Dino|   5|
| Maserati Bora|   5|
+--------------+----+



## That's it!
You successfully completed this notebook! You learned how to load a DataFrame, view and filter the data, aggregate the data, perform operations on the data in specific columns, and run SQL queries against the data. For more information about Spark, see the [Spark Quick Start Guide](http://spark.apache.org/docs/latest/quick-start.html).

### Authors

**Saeed Aghabozorgi**, PhD, is a Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge. He is a researcher in the data mining field and an expert in developing advanced analytic methods like machine learning and statistical modelling on large data sets.

**Polong Lin** is a Data Scientist at IBM in Canada. Under the Emerging Technologies division, Polong is responsible for educating the next generation of data scientists through Big Data University. Polong is a regular speaker in conferences and meetups, and holds an M.Sc. in Cognitive Psychology.

Copyright © 2016, 2018 Big Data University. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/" rel="noopener noreferrer" target="_blank">MIT License</a>.